In [1]:
import tensorflow as tf

In [2]:
x=tf.Variable(3,name="x")
y=tf.Variable(4,name="y")
f=x*x*y+y+2


In [4]:
# the shortcut
init=tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result=f.eval()
result

42

In [5]:
# linear regression
import numpy as np
from sklearn.datasets import fetch_california_housing

In [85]:
housing=fetch_california_housing()
m,n=housing.data.shape
housing_data_plus_bias=np.c_[np.ones((m,1)),housing.data]
housing_data_plus_bias
housing.target

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

In [87]:
scaled_housing_data_plus_bias

array([[ 0.        ,  2.34476576,  0.98214266, ..., -0.04959654,
         1.05254828, -1.32783522],
       [ 0.        ,  2.33223796, -0.60701891, ..., -0.09251223,
         1.04318455, -1.32284391],
       [ 0.        ,  1.7826994 ,  1.85618152, ..., -0.02584253,
         1.03850269, -1.33282653],
       ...,
       [ 0.        , -1.14259331, -0.92485123, ..., -0.0717345 ,
         1.77823747, -0.8237132 ],
       [ 0.        , -1.05458292, -0.84539315, ..., -0.09122515,
         1.77823747, -0.87362627],
       [ 0.        , -0.78012947, -1.00430931, ..., -0.04368215,
         1.75014627, -0.83369581]])

In [90]:
from sklearn.preprocessing import scale
scaled_housing_data_plus_bias = scale( housing_data_plus_bias, axis=0, with_mean=True, with_std=True, copy=True )
X=tf.constant(housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
XT=tf.transpose(X)
theta=tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)
with tf.Session() as sess:
    theta_value=theta.eval()
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

In [9]:
# implementing the gradient descent

In [69]:
# mannually calculate
from tqdm import tqdm
n_epochs=10000
learning_rate=0.01
from sklearn.preprocessing import scale
scaled_housing_data_plus_bias = scale( housing_data_plus_bias, axis=0, with_mean=True, with_std=True, copy=True )
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")
y_pred=tf.matmul(X,theta,name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
gradients=2/m*tf.matmul(tf.transpose(X),error)
training_op=tf.assign(theta,theta-learning_rate*gradients)
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in tqdm(range(n_epochs)):
        sess.run(training_op)
    best_theta=theta.eval()
    print(best_theta)
        

100%|██████████| 10000/10000 [00:02<00:00, 4664.84it/s]


[[-0.76921654]
 [ 0.8296334 ]
 [ 0.11875438]
 [-0.26555124]
 [ 0.30571607]
 [-0.00450224]
 [-0.03932684]
 [-0.89985305]
 [-0.87050986]]


In [97]:
# use autodiff
from tqdm import tqdm
n_epochs=10000
learning_rate=0.01
from sklearn.preprocessing import scale
scaled_housing_data_plus_bias = scale( housing_data_plus_bias, axis=0, with_mean=True, with_std=True, copy=True )
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")
y_pred=tf.matmul(X,theta,name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
gradients=tf.gradients(mse,[theta])[0]
training_op=tf.assign(theta,theta-learning_rate*gradients)
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in tqdm(range(n_epochs)):
        sess.run(training_op)
    best_theta=theta.eval()
    print(best_theta)


100%|██████████| 10000/10000 [00:02<00:00, 3954.00it/s]


[[ 0.13630438]
 [ 0.8296331 ]
 [ 0.11875448]
 [-0.26555073]
 [ 0.3057157 ]
 [-0.00450223]
 [-0.03932685]
 [-0.8998542 ]
 [-0.87051094]]


In [96]:
# use optimizer
from tqdm import tqdm
n_epochs=10000
learning_rate=0.01
from sklearn.preprocessing import scale
scaled_housing_data_plus_bias = scale( housing_data_plus_bias, axis=0, with_mean=True, with_std=True, copy=True )
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")
y_pred=tf.matmul(X,theta,name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
#optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
training_op=optimizer.minimize(mse)
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in tqdm(range(n_epochs)):
        sess.run(training_op)
    best_theta=theta.eval()
    print(best_theta)

100%|██████████| 10000/10000 [00:02<00:00, 4174.78it/s]


[[-0.69185996]
 [ 0.82962126]
 [ 0.11875215]
 [-0.26553   ]
 [ 0.30569866]
 [-0.00450288]
 [-0.03932635]
 [-0.89988035]
 [-0.8705356 ]]
